Exporting Weather and Jetfuel Price Data

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from FAA import Ohare
from FAA import Hanger
import pandas as pd

In [3]:
os.getcwd()

'/Users/bjg/Metis/Projects/McNulty/FlightOnTime/notebooks'

# Export Weather Data to SQL

In [5]:
pd.read_csv('../data/weather/NOAA_ORD_WS_2017_18.csv').columns

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD',
       'TAVG', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09'],
      dtype='object')

In [13]:
ord_2017_oct18_weather.head(2)

,DATE,TAVG,AWND,PRCP,SNOW,SNWD,NAME,STATION
0,2017-01-01,27,4.92,0.00,0.0,0.0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",USW00094846
1,2017-01-02,34,6.26,0.11,0.0,0.0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",USW00094846


In [10]:
ord_2017_oct18_weather = pd.read_csv('../data/weather/NOAA_ORD_WS_2017_18.csv')[['DATE', 'TAVG', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'NAME', 'STATION']]

In [12]:
from sqlalchemy import create_engine

username = 'bjg'

params = { 'host': 'localhost',
           'port': 5432
}

connection_string = f'postgres://{username}:{params["host"]}@{params["host"]}:{params["port"]}/faa'
engine = create_engine(connection_string)

In [14]:
ord_2017_oct18_weather.to_sql('WEATHER_ORD', engine, index=False)

# Export Jetfuel Prices to SQL

In [18]:
jf_prices = pd.read_csv('../data/jet_fl_price/WJFUELUSGULF.csv')

In [21]:
jf_prices.head()

,DATE,WJFUELUSGULF
0,2017-01-06,1.539
1,2017-01-13,1.506
2,2017-01-20,1.505
3,2017-01-27,1.513
4,2017-02-03,1.518


In [24]:
jf_prices.DATE = pd.to_datetime(jf_prices.DATE)

In [96]:
jf_prices.head()

,DATE,WJFUELUSGULF
0,2017-01-06,1.539
1,2017-01-13,1.506
2,2017-01-20,1.505
3,2017-01-27,1.513
4,2017-02-03,1.518


In [98]:
jf_prices.to_sql('JETFUEL_PRICE', engine, index=False)

# Trying to create daily JF Prices, pivoting to starto fo week column in SQL

In [38]:
from datetime import timedelta

In [47]:
yesterday = timedelta(days=-1)

jf_prices.loc[0].DATE 

Timestamp('2017-01-06 00:00:00')

In [48]:
jf_prices.loc[0].DATE + extra_time

Timestamp('2017-01-05 00:00:00')

In [67]:
def last_week(date):
    week = [date]
    
    yesterday = timedelta(days=-1)
    
    for i in range(6):
        
        week.append(date + yesterday)
        yesterday = yesterday + timedelta(days=-1)
    
    return week

In [69]:
import pprint
pprint.pprint(last_week(jf_prices.loc[1].DATE))

[Timestamp('2017-01-13 00:00:00'),
 Timestamp('2017-01-12 00:00:00'),
 Timestamp('2017-01-11 00:00:00'),
 Timestamp('2017-01-10 00:00:00'),
 Timestamp('2017-01-09 00:00:00'),
 Timestamp('2017-01-08 00:00:00'),
 Timestamp('2017-01-07 00:00:00')]


In [70]:
jf_prices.loc[0]

DATE            2017-01-06 00:00:00
WJFUELUSGULF                  1.539
Name: 0, dtype: object

In [92]:
def aply_last_wk(price_date):
    
    week = last_week(price_date['DATE'])
    week.append(price_date['WJFUELUSGULF'])
    
    return pd.Series(data=week, index=pd.Index(['DATE', 'DATE', 'DATE', 'DATE', 'DATE', 'DATE', 'DATE', 'WJFUELUSGULF']))

In [95]:
aply_last_wk(jf_prices.loc[1])

DATE            2017-01-13 00:00:00
DATE            2017-01-12 00:00:00
DATE            2017-01-11 00:00:00
DATE            2017-01-10 00:00:00
DATE            2017-01-09 00:00:00
DATE            2017-01-08 00:00:00
DATE            2017-01-07 00:00:00
WJFUELUSGULF                  1.506
dtype: object

In [94]:
jf_prices.apply(aply_last_wk)

KeyError: ('DATE', 'occurred at index DATE')